# <span style="color:darkblue">netivreg: Estimation of Peer Efects in Endogenous Social Networks</span> 

Pablo Estrada, Juan Estrada, Kim P. Huynh, David T. Jacho-Chavez and Leonardo Sanchez-Aragon

<font size="3"> 
    
The command **netivreg** implements the following estimators for endogenous linear-in-means models:
    
- Generalized Two-Stage Least Squares (G2SLS)
- Generalized Three-Stage Least Squares (G3SLS)
- Generalized Method of Moments (GMM)

# Start Stata from Python

In [2]:
import stata_setup
stata_setup.config('/Applications/Stata','mp')   # for macOS
# stata_setup.config('/usr/local/stata17','mp')   # for ubuntu


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706303466
  Licensed to: Pablo Estrada
               Emory University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


# Simulated Data

<font size="3"> 
    
We use the following version of the linear-in-means model:

$$
\begin{align}
y_{i}=&1+0.7\sum_{j=1}^n\overline{w}_{ij}y_{j}+0.33\sum_{j=1}^n\overline{w}_{ij}x_{1i}+0.33\sum_{j=1}^n\overline{w}_{ij}x_{2i}+0.33\sum_{j=1}^n\overline{w}_{ij}x_{3i}\nonumber\\ 
 &+0.33x_{1i}+0.33x_{2i}+0.33x_{3i}+v_{i}\text{,}\label{sim_equation}
\end{align}
$$

where $x_{ki}$ are drawn from an i.i.d. $\sim N(0,3)$ for $k=1,2,3$, which are independent of each other. The weights $\overline{w}_{ij}$ are row-normalized versions of the adjacency matrix $\mathbf{W}=[w_{ij}]$, i.e., $\overline{w}_{ij}=w_{ij}/\sum_{j=1}^n w_{ij}$. 
    
The $\mathbf{W}$ adjacency matrix is generated from $\mathbf{W}_0=[w_{0;ij}]$ which in turn is generated from random graph with a density of 0.01.

In [3]:
%%stata
use ../data/stata/data_sim.dta, replace
format y_endo y_exo x1 x2 x3 x4 %9.3f
list in 1/5, table 


. use ../data/stata/data_sim.dta, replace

. format y_endo y_exo x1 x2 x3 x4 %9.3f

. list in 1/5, table 

     +--------------------------------------------------------+
     | id   y_exo   y_endo       x1      x2       x3       x4 |
     |--------------------------------------------------------|
  1. |  1   4.072    4.555   -0.523   0.926    2.136   -0.546 |
  2. |  2   4.584    4.665    2.611   1.455   -0.926    0.759 |
  3. |  3   3.887    3.671    3.125   0.513   -2.718   -2.132 |
  4. |  4   3.736    3.962   -2.674   1.504    1.769    0.091 |
  5. |  5   6.360    7.002   -0.993   0.345    1.126    1.120 |
     +--------------------------------------------------------+

. 


In [4]:
%%stata
use ../data/stata/W_sim.dta, replace
list in 113/117, table 


. use ../data/stata/W_sim.dta, replace

. list in 113/117, table 

     +-----------------+
     | source   target |
     |-----------------|
113. |     28      259 |
114. |     28      361 |
115. |     29       67 |
116. |     29       79 |
117. |     29      196 |
     +-----------------+

. 


In [5]:
%%stata
use ../data/stata/W0_sim.dta, replace
list in 113/117, table 


. use ../data/stata/W0_sim.dta, replace

. list in 113/117, table 

     +-----------------+
     | source   target |
     |-----------------|
113. |     30      167 |
114. |     30      325 |
115. |     31       38 |
116. |     31       83 |
117. |     31      132 |
     +-----------------+

. 


# 1. G2SLS

## 1.1. Assumptions

<br>

<font size="3">

1. No correlated effects: $\mathbb{E}[\mathbf{v}|\mathbf{X}, \mathbf{W}] = 0$
    
2. Relevance: $\mathbf{I}$, $\mathbf{W}$, $\mathbf{W}^{2}$ and $\gamma\beta + \delta \neq 0$

## 1.2. Estimation

<br>

<font size="3">

- **Step 1:**
    
Regress $\mathbf{y}$ on $\boldsymbol{\iota}$, $\mathbf{X}$, $\mathbf{W}\mathbf{y}$, and $\mathbf{W}\mathbf{X}$ by 2SLS using $\boldsymbol{\iota}$, $\mathbf{X}$, $\mathbf{W}^{2}\mathbf{X}$, and $\mathbf{W}\mathbf{X}$ as instruments. 
    
- **Step 2:**

Regress $\mathbf{y}$ on $\boldsymbol{\iota}$, $\mathbf{X}$, and $[\widehat{\mathbf{W}\mathbf{y}} \quad \widehat{\mathbf{W}\mathbf{X}}]$ by IV using $\boldsymbol{\iota}$, $\mathbf{X}$, and $[\widehat{\mathbf{Z}} \quad \widehat{\mathbf{W}\mathbf{X}}]$ as instruments, where $\widehat{\mathbf{Z}}$ is the optimal instrument.

In [6]:
%%stata
use ../data/stata/data_sim.dta, replace
frame create edges
frame edges: use ../data/stata/W_sim.dta
netivreg g3sls y_exo x1 x2 x3 x4 (edges = edges) 

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 176, in run
    raise SystemError(output)
SystemError: 
. use ../data/stata/data_sim.dta, replace

. frame create edges

. frame edges: use ../data/stata/W_sim.dta

. netivreg g3sls y_exo x1 x2 x3 x4 (edges = edges) 
command netivreg is unrecognized
r(199);
r(199);



In [7]:
%%stata
netivreg g3sls y_endo x1 x2 x3 x4 (edges = edges) 

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 169, in run
    raise SystemError(output)
SystemError: command netivreg is unrecognized
r(199);



# 2. G3SLS

## 2.1. Assumptions

<br>

<font size="3">

1. No correlated effects: $\mathbb{E}[\mathbf{v}|\mathbf{X}, \mathbf{W}_0] = 0$
    
2. Relevance: Relevance: $\mathbf{I}$, $\mathbf{W}_{0}$, $\mathbf{W}_{0}^{2}$ and $\gamma\boldsymbol{\pi}_{1}^{\top}\theta+\boldsymbol{\pi}_{2}^{\top}\theta \neq 0$

## 2.2. Estimation

<br>

<font size="3">

- **Step 1:**

Regress $\mathbf{W}\mathbf{S}$ on $\mathbf{W}_0\mathbf{S}$ by OLS. Get $\widehat{\mathbf{W}\mathbf{S}}$ and $\widehat{\mathbf{U}}$.
    
    
    
- **Step 2:**
    
    
Regress $\mathbf{y}$ on $\boldsymbol{\iota}$, $\mathbf{X}$, $\mathbf{W}_{0}\mathbf{y}$, and $\mathbf{W}_{0}\mathbf{X}$ by 2SLS using $\boldsymbol{\iota}$, $\mathbf{X}$, $\mathbf{W}_{0}^{2}\mathbf{X}$, and $\mathbf{W}_{0}\mathbf{X}$ as instruments. From the equation 
    
    
$$\widehat{\alpha}_{\text{2SLS}}\boldsymbol{\iota}+\widehat{\theta}^\ast_{\text{2SLS}}\mathbf{W}_{0}\mathbf{S}+\widehat{\gamma}_{\text{2SLS}}\mathbf{X},$$
    
    
    
get the parameters $[\widehat{\alpha}_{\text{2SLS}},\widehat{\gamma}_{\text{2SLS}},\widehat{\theta}_{\text{2SLS}}]^{\top}$ where $\widehat{\boldsymbol{\theta}}_{\text{2SLS}} = \widehat{\Pi}^{-1}\widehat{\theta}^\ast_{\text{2SLS}}$.
    
    
- **Step 3:**
    
    

Regress $\mathbf{y}$ on $\boldsymbol{\iota}$, $\mathbf{X}$, and $\widehat{\mathbf{W}\mathbf{S}}$ by IV using $\boldsymbol{\iota}$, $\mathbf{X}$, and $[\widehat{\mathbf{Z}}\quad \mathbf{W}_{0}\mathbf{X}]\widehat{\boldsymbol{\Pi}}$ as instruments, where $\widehat{\mathbf{Z}}=\mathbf{W}_{0}\left(\mathbf{I}-(\widehat{\boldsymbol{\pi}}_1^\top\widehat{\boldsymbol{\theta}}_{\text{2SLS}})\mathbf{W}_{0}\right)^{-1}\left\{\widehat{\alpha}_{\text{2SLS}}\boldsymbol{\iota}+\left(\widehat{\gamma}_{\text{2SLS}}\mathbf{I}+(\widehat{\boldsymbol{\pi}}_2^\top\widehat{\boldsymbol{\theta}}_{\text{2SLS}})\mathbf{W}_{0}\right)\mathbf{X}\right\}$. From the equation
    
    
    
$$\widehat{\alpha}_{\text{G3SLS}}\boldsymbol{\iota}+\widehat{\theta}_{\text{G3SLS}}\mathbf{W}\mathbf{S}+\widehat{\gamma}_{\text{G3SLS}}\mathbf{X},$$
    
    
    
get the parameters $[\widehat{\alpha}_{\text{G3SLS}},\widehat{\gamma}_{\text{G3SLS}},\widehat{\beta}_{\text{G3SLS}},\widehat{\delta}_{\text{G3SLS}}]^{\top}$ and the residuals $\widehat{\mathbf{v}}\equiv\mathbf{y}-\widehat{\alpha}_{\text{G3SLS}}\boldsymbol{\iota}-\widehat{\gamma}_{\text{G3SLS}}\mathbf{X}-\widehat{\beta}_{\text{G3SLS}}\mathbf{W}\mathbf{y}-\widehat{\delta}_{\text{G3SLS}}\mathbf{W}\mathbf{X}$.

In [8]:
%%stata
frame create edges0
frame edges0: use ../data/stata/W0_sim.dta
netivreg g3sls y_endo x1 x2 x3 x4 (edges = edges0)

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 176, in run
    raise SystemError(output)
SystemError: 
. frame create edges0

. frame edges0: use ../data/stata/W0_sim.dta

. netivreg g3sls y_endo x1 x2 x3 x4 (edges = edges0)
command netivreg is unrecognized
r(199);
r(199);



In [9]:
%%stata
netivreg g3sls y_endo x1 x2 x3 x4 (edges = edges0), first

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 169, in run
    raise SystemError(output)
SystemError


In [10]:
%%stata
netivreg g3sls y_endo x1 x2 x3 x4 (edges = edges0), second

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 169, in run
    raise SystemError(output)
SystemError: command netivreg is unrecognized
r(199);



# 3. GMM

## 3.1. Assumptions

<br>

<font size="3">
    
We define the matrices $\mathbf{D}=[\mathbf{W}\mathbf{y}, \mathbf{W}\mathbf{X}, \widetilde{\mathbf{X}}]$ to be the matrix of regressors of the linear-in-means model and $\mathbf{Z}=[\mathbf{W}_{N,0}^{p}\mathbf{X},\mathbf{W}_{N,0}^{p-1}\mathbf{X},\dots,\mathbf{W}_{N,0}\mathbf{X}, \widetilde{\mathbf{X}}]$ to be the matrix of *instruments*.

1. No correlated effects: $\mathbb{E}[\mathbf{v}|\mathbf{X}, \mathbf{W}_0] = 0$    
2. the conditional probability $\mathcal{F}(\mathcal{G}, \mathbf{v}\mid \mathcal{G}_{0},\mathbf{X})$ is such that $\Pr(w_{i,j}>0|\mathcal{G}_{0},\mathbf{X})=\rho(w_{0;i,j},\mathcal{G}_{0},\mathbf{X},\mathbf{v})$
3. The matrix $\mathbb{E}[\sum_{i \in \mathcal{I}_{N}}\mathbf{z}_{i}\mathbf{d}_{i}^{\top}]<\infty$ has full column rank.

## 3.2. Estimation

<br>

<font size="3">

Standard linear GMM estimator:

$$\widehat{\boldsymbol{\psi}}_{\text{GMM}} =  [\mathbf{D}_{n}^{\top}\mathbf{Z}_{n}\mathbf{A}_{n} \mathbf{Z}_{n}^{\top}\mathbf{D}_{n}]^{-1} [\mathbf{D}_{n}^{\top}\mathbf{Z}_{n}\mathbf{A}_{n}\mathbf{Z}_{n}^{\top}\mathbf{y}_{n}]$$

where the full rank weighting matrix $\mathbf{A}_{n}$ is assumed to converge in probability to $\mathbf{A}$

In [11]:
%%stata
netivreg gmm y_endo x1 x2 x3 x4 (edges = edges0)

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 169, in run
    raise SystemError(output)
SystemError: command netivreg is unrecognized
r(199);



# Real Data

In [12]:
%%stata
use data/articles.dta, replace
describe


. use data/articles.dta, replace
(Data on articles published in the aer, eca, jpe, & qje between 2000-2002)

. describe

Contains data from data/articles.dta
 Observations:           729                  Data on articles published in
                                                the aer, eca, jpe, & qje
                                                between 2000-2002
    Variables:            12                  12 Sep 2020 14:09
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
id              int     %9.0g                 Article unique identifier
lcitations      float   %9.0g                 Log of total citations 8 years
                                                post publication
editor          int     %8.0g                 1 if at least one of the
               

In [13]:
%%stata
gen citations = exp(lcitations)
tabulate journal year, summarize(citations)


. gen citations = exp(lcitations)

. tabulate journal year, summarize(citations)

          Means, Standard Deviations and Frequencies of citations

Journal=ae |
r,eca,jpe, |      Year=2000,2001,2002
       qje |      2000       2001       2002 |     Total
-----------+---------------------------------+----------
       aer | 52.417721  54.931821  48.652174 | 51.934364
           | 73.653308  90.712233  49.893607 | 72.800192
           |        79         88         92 |       259
-----------+---------------------------------+----------
       eca | 49.627451  43.328125  37.177779 | 42.195122
           | 52.045351  51.688336  43.565161 | 48.397466
           |        51         64         90 |       205
-----------+---------------------------------+----------
       jpe | 34.530612  32.863637  46.666667 | 38.141844
           | 26.257619  30.229811  35.717172 | 31.362075
           |        49         44         48 |       141
-----------+---------------------------------+----------
 

In [14]:
%%stata
summarize editor diff_gender isolated n_pages n_authors n_references


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
      editor |        729    .0452675     .208033          0          1
 diff_gender |        729    .1303155    .3368814          0          1
    isolated |        729    .5281207    .4995513          0          1
     n_pages |        729    25.15775    11.53631          3         76
   n_authors |        729    1.888889    .7486251          1          5
-------------+---------------------------------------------------------
n_references |        729    31.40329    17.84755          0        177


In [15]:
%%stata
frame reset
frame create edges
frame edges: use data/edges.dta
frame edges: list in 1/5, table 


. frame reset

. frame create edges

. frame edges: use data/edges.dta
(Co-authorship network among articles published in the aer, eca, jpe, & qje bet
> we)

. frame edges: list in 1/5, table 

     +-----------------+
     | source   target |
     |-----------------|
  1. |      4      472 |
  2. |      5      221 |
  3. |      5      463 |
  4. |      5      478 |
  5. |      5      665 |
     +-----------------+

. 


In [16]:
%%stata
frame create edges0
frame edges: use data/edges0.dta
frame edges: list in 1/5, table 


. frame create edges0

. frame edges: use data/edges0.dta
(Alumni network among articles published in the aer, eca, jpe, & qje between 20
> 00)

. frame edges: list in 1/5, table 

     +-----------------+
     | source   target |
     |-----------------|
  1. |      2      482 |
  2. |      2      534 |
  3. |      4      129 |
  4. |      4      136 |
  5. |      4      407 |
     +-----------------+

. 


In [17]:
%%stata
use data/articles.dta
tabulate journal, g(journal)
tabulate year, g(year)


. use data/articles.dta
(Data on articles published in the aer, eca, jpe, & qje between 2000-2002)

. tabulate journal, g(journal)

Journal=aer |
,eca,jpe,qj |
          e |      Freq.     Percent        Cum.
------------+-----------------------------------
        aer |        259       35.53       35.53
        eca |        205       28.12       63.65
        jpe |        141       19.34       82.99
        qje |        124       17.01      100.00
------------+-----------------------------------
      Total |        729      100.00

. tabulate year, g(year)

Year=2000,2 |
   001,2002 |      Freq.     Percent        Cum.
------------+-----------------------------------
       2000 |        221       30.32       30.32
       2001 |        238       32.65       62.96
       2002 |        270       37.04      100.00
------------+-----------------------------------
      Total |        729      100.00

. 


## Real Data Analysis 

<br>

<font size="3">
    
    
$$
\begin{align}
  y_{i,r,t} ={}&\alpha+\beta\sum_{j \neq i}w_{i,j}y_{j,r,t}+\sum_{j \neq i}w_{i,j}\mathbf{x}_{j,r,t}^{\top}\boldsymbol{\delta}+ \mathbf{x}_{i,r,t}^{\top}\boldsymbol{\gamma}\nonumber\\
  & +\lambda_{r}+\lambda_{t}+v_{i,r,t}\text{,}
  \label{modelo}
\end{align}
$$

- $y_{i,r,t}$ represents the natural logarithm of article $i$'s citations eight years post publication (lcitations) in journal $r$ in year $t$.
- $\mathbf{x}_{j,r,t}$ includes diff_gender and editor of article $j$ in journal $r$ in year $t$.
- $\mathbf{x}_{i,r,t}$ include the same characteristics for article $i$ plus its number of pages (n_pages), authors (n_authors), bibliographic references (n_references), and whether or not it shares a co-author relationship with other articles (isolated).
- Fixed effects include journal ($\lambda_{r}$) and year ($\lambda_t$).
- Co-authors' network ($\mathbf{W}$) is endogenous and the alumni network ($\mathbf{W_0}$) is pre-determined and is therefore assumed to be exogenous.

In [18]:
%%stata
use ../data/stata/articles.dta, replace
describe


. use ../data/stata/articles.dta, replace
(Data on articles published in the aer, eca, jpe, & qje between 2000-2002)

. describe

Contains data from ../data/stata/articles.dta
 Observations:           729                  Data on articles published in
                                                the aer, eca, jpe, & qje
                                                between 2000-2002
    Variables:            12                  12 Sep 2020 14:09
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
id              int     %9.0g                 Article unique identifier
lcitations      float   %9.0g                 Log of total citations 8 years
                                                post publication
editor          int     %8.0g                 1 if at least one of t

In [19]:
%%stata
gen citations = exp(lcitations)
tabulate journal year, summarize(citations)


. gen citations = exp(lcitations)

. tabulate journal year, summarize(citations)

          Means, Standard Deviations and Frequencies of citations

Journal=ae |
r,eca,jpe, |      Year=2000,2001,2002
       qje |      2000       2001       2002 |     Total
-----------+---------------------------------+----------
       aer | 52.417721  54.931821  48.652174 | 51.934364
           | 73.653308  90.712233  49.893607 | 72.800192
           |        79         88         92 |       259
-----------+---------------------------------+----------
       eca | 49.627451  43.328125  37.177779 | 42.195122
           | 52.045351  51.688336  43.565161 | 48.397466
           |        51         64         90 |       205
-----------+---------------------------------+----------
       jpe | 34.530612  32.863637  46.666667 | 38.141844
           | 26.257619  30.229811  35.717172 | 31.362075
           |        49         44         48 |       141
-----------+---------------------------------+----------
 

In [20]:
%%stata
summarize editor diff_gender isolated n_pages n_authors n_references


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
      editor |        729    .0452675     .208033          0          1
 diff_gender |        729    .1303155    .3368814          0          1
    isolated |        729    .5281207    .4995513          0          1
     n_pages |        729    25.15775    11.53631          3         76
   n_authors |        729    1.888889    .7486251          1          5
-------------+---------------------------------------------------------
n_references |        729    31.40329    17.84755          0        177


In [21]:
%%stata
frame create edges1
frame edges1: use ../data/stata/edges.dta
frame edges1: list in 1/5, table 


. frame create edges1

. frame edges1: use ../data/stata/edges.dta
(Co-authorship network among articles published in the aer, eca, jpe, & qje bet
> we)

. frame edges1: list in 1/5, table 

     +-----------------+
     | source   target |
     |-----------------|
  1. |      4      472 |
  2. |      5      221 |
  3. |      5      463 |
  4. |      5      478 |
  5. |      5      665 |
     +-----------------+

. 


In [22]:
%%stata
frame create edges2
frame edges2: use ../data/stata/edges0.dta
frame edges2: list in 1/5, table 


. frame create edges2

. frame edges2: use ../data/stata/edges0.dta
(Alumni network among articles published in the aer, eca, jpe, & qje between 20
> 00)

. frame edges2: list in 1/5, table 

     +-----------------+
     | source   target |
     |-----------------|
  1. |      2      482 |
  2. |      2      534 |
  3. |      4      129 |
  4. |      4      136 |
  5. |      4      407 |
     +-----------------+

. 


In [23]:
%%stata
use ../data/stata/articles.dta, replace
tabulate journal, g(journal)
tabulate year, g(year)


. use ../data/stata/articles.dta, replace
(Data on articles published in the aer, eca, jpe, & qje between 2000-2002)

. tabulate journal, g(journal)

Journal=aer |
,eca,jpe,qj |
          e |      Freq.     Percent        Cum.
------------+-----------------------------------
        aer |        259       35.53       35.53
        eca |        205       28.12       63.65
        jpe |        141       19.34       82.99
        qje |        124       17.01      100.00
------------+-----------------------------------
      Total |        729      100.00

. tabulate year, g(year)

Year=2000,2 |
   001,2002 |      Freq.     Percent        Cum.
------------+-----------------------------------
       2000 |        221       30.32       30.32
       2001 |        238       32.65       62.96
       2002 |        270       37.04      100.00
------------+-----------------------------------
      Total |        729      100.00

. 


In [24]:
%%stata
netivreg g3sls lcitations editor diff_gender n_pages n_authors n_references ///
isolated journal2-journal4 year2-year3 (edges1 = edges2), ///
wx(editor diff_gender) cluster(c_coauthor)

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 176, in run
    raise SystemError(output)
SystemError: 
. netivreg g3sls lcitations editor diff_gender n_pages n_authors n_references /
> //
> isolated journal2-journal4 year2-year3 (edges1 = edges2), ///
> wx(editor diff_gender) cluster(c_coauthor)
command netivreg is unrecognized
r(199);
r(199);



In [25]:
%%stata
netivreg gmm lcitations editor diff_gender n_pages n_authors n_references /// 
isolated journal2-journal4 year2-year3 (edges1 = edges2), wx(editor diff_gender) ///
wz(editor diff_gender n_pages n_authors n_references isolated) maxp(4)

Exception in thread Stata:
Traceback (most recent call last):
  File "/Users/pablo/opt/anaconda3/envs/env_netivreg/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 176, in run
    raise SystemError(output)
SystemError: 
. netivreg gmm lcitations editor diff_gender n_pages n_authors n_references ///
>  
> isolated journal2-journal4 year2-year3 (edges1 = edges2), wx(editor diff_gend
> er) ///
> wz(editor diff_gender n_pages n_authors n_references isolated) maxp(4)
command netivreg is unrecognized
r(199);
r(199);

